In [17]:
# https://google.github.io/adk-docs/sessions/state/#organizing-state-with-prefixes-scope-matters

state1 = 'test'

formatting_instructions = f"""
three {{{state1}}} # take it from the .state; it is mandatory
three? {{{state1}?}} # take it from the .state; it is optional
two {{state1}} # consider it as exact value in brackets
one {state1} ## consider it as a simple variable defined somewhere else
"""
print(formatting_instructions)


three {test} # take it from the .state; it is mandatory
three? {test?} # take it from the .state; it is optional
two {state1} # consider it as exact value in brackets
one test ## consider it as a simple variable defined somewhere else

{test?}



In [20]:
"{{{}?}}".format(state1)

'{test?}'

In [21]:
# NEXT STEPS:
## V1 ##
# - test if works (when wrong movie is asked for)
# - create a workflow diagram + vid and short desc
# - focus on the srcaping agent which doesn't know how to scrap vod STREAMING only

## V2 ##
# - deploy
# - add more movies at the same time/ parallel
## V3 ##
# - adding before_agent_callback = increment_attempt instead of the tool usage